In [13]:
import pandas as pd
import numpy as np

import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col

In [14]:
df  =  pd.read_excel("UYdata.xlsx")

In [15]:
print(df.shape)
print(df.dtypes)

(3003, 90)
Unnamed: 0             int64
villagecode_str        int64
google_km            float64
tehsil_e             float64
treatment            float64
                      ...   
treatment12          float64
hhhead_mrtlstatus      int64
firewood_qty_b       float64
dung_qty_b           float64
_merge                 int64
Length: 90, dtype: object


In [16]:
df.head()

,Unnamed: 0,villagecode_str,google_km,tehsil_e,treatment,noncompliance_village,treatment012,treatment_h,treatment_hs,hhscr_8_b,...,monthly_sumb,monthly_wetb,att_missing,treatment01,treatment02,treatment12,hhhead_mrtlstatus,firewood_qty_b,dung_qty_b,_merge
0,1,475907,26.152000,1.0,2.0,0,1,0,1,0,...,0.000000,0.00,0,NaN,1.0,1.0,1,0.0,0.0,3
1,2,475911,28.915001,1.0,2.0,0,1,0,1,0,...,0.000000,0.00,0,NaN,1.0,1.0,1,0.0,0.0,3
2,3,475913,30.469000,1.0,2.0,0,1,0,1,1,...,0.333333,0.00,0,NaN,1.0,1.0,1,0.0,0.0,3
3,4,475914,32.390999,1.0,2.0,0,1,0,1,0,...,0.000000,0.00,0,NaN,1.0,1.0,1,0.0,500.0,3
4,5,475915,28.500000,1.0,2.0,0,1,0,1,1,...,0.666667,0.25,0,NaN,1.0,1.0,1,0.0,0.0,3


In [17]:
# total number of villages: 150
# each village has 20-21 observations
# number of villages in each category: C(0.0), H(1.0), H+S(2.0)
# need to know this information for culstering standard errors
df.groupby("treatment")["villagecode_str"].nunique()

treatment
0.0    50
1.0    46
2.0    46
Name: villagecode_str, dtype: int64

In [18]:
# to understand each column better, value counts
for col in df.columns[2:]:
    print(df[col].value_counts(normalize=True).round(2).astype(str) + ' %' +
          " " + 
          (df[col].value_counts().astype(str))
         )
    print("\n")
    print("###----------------------------------------###")
    print("\n")

9.995     0.01 % 40
6.894     0.01 % 21
19.737    0.01 % 21
12.073    0.01 % 21
15.481    0.01 % 20
            ...    
12.216    0.01 % 20
21.666    0.01 % 20
14.908    0.01 % 20
14.271    0.01 % 20
18.539    0.01 % 20
Name: google_km, Length: 149, dtype: object


###----------------------------------------###


1.0    0.37 % 1101
4.0     0.29 % 866
5.0     0.28 % 829
2.0     0.04 % 113
3.0      0.01 % 40
Name: tehsil_e, dtype: object


###----------------------------------------###


0.0    0.35 % 1000
2.0     0.32 % 922
1.0     0.32 % 921
Name: treatment, dtype: object


###----------------------------------------###


0    0.95 % 2843
1     0.05 % 160
Name: noncompliance_village, dtype: object


###----------------------------------------###


1    0.61 % 1843
0    0.39 % 1160
Name: treatment012, dtype: object


###----------------------------------------###


0    0.69 % 2082
1     0.31 % 921
Name: treatment_h, dtype: object


###----------------------------------------###


0    

## Control Variable names:

1. Household size: hhnum_b
2. Female headed household: hhgender_1_b
3. Age of primary cook: age_pc_b
4. household head education: edu_hhhead_b
5. primary cook edu: pc_edu_b
6. Household head is married: hhhead_mrtlstatus
7. HH occupation: occu_b
8. SC/ST:  scst_b
9. OBC: obcpop_b
10. hindu: hindu_b
11. hh wealth : assets_index
12. Asha: ashacred_b

In [20]:
# filtering out household characteristics
hh_char = df[["villagecode_str","treatment","tehsil_e","hhnum_b", "hhgender_1_b", "age_pc_b", "edu_hhhead_b", 
                    "pc_edu_b", "hhhead_mrtlstatus", "occu_b",  "scst_b", 
                    "obcpop_b", "hindu_b", "assets_index", "ashacred_b", "att_missing"]]

In [21]:
# remove observations with attribution missing
hh_char_clean = hh_char.loc[hh_char.att_missing == 0]
# check columns just to be sure
hh_char_clean.columns

Index(['villagecode_str', 'treatment', 'tehsil_e', 'hhnum_b', 'hhgender_1_b',
       'age_pc_b', 'edu_hhhead_b', 'pc_edu_b', 'hhhead_mrtlstatus', 'occu_b',
       'scst_b', 'obcpop_b', 'hindu_b', 'assets_index', 'ashacred_b',
       'att_missing'],
      dtype='object')

In [22]:
# check to see if any village has dropped. 
hh_char_clean.groupby("treatment")["villagecode_str"].nunique()
# None have dropped

treatment
0.0    50
1.0    46
2.0    46
Name: villagecode_str, dtype: int64

In [23]:
# check to see how many obvs have dropped from each category
hh_char_clean.groupby("treatment")["villagecode_str"].count()
# the numbers match the table numbers 

treatment
0.0    982
1.0    907
2.0    902
Name: villagecode_str, dtype: int64

In [24]:
#### Steps in writing the below function: ####

# get dependent variables first (covariates) as columns
# calculate mean and sample standard deviation using groupby
# for each characteristic we heave to run 2 seperate regressions each:
#         1. treatment 2 dummies for diff with C as the reference to get C-H, C-(H+S)
#         2. treatment 2 dummies for diff with H as the reference to get H-(H+S)


def balance_table(dataframe, treatment, cluster, mean_or_sde):

    '''
    dataframe: pandas dataframe where the columns are the characteristics (covariates)
               the dataframe should also have a treatment column and a cluster id column 
    treatment: the name of the column in the dataframe with control 
               coded as 0, treatment 1 as 1, treatment 2 as 2
    cluster: the name of the column in the dataframe with unique cluster ids. 
             for ex village id, firm id, etc.
    mean_or_sde: for mean input "mean" and for standard error input "sde" 
    
    returns a generator of the paramters of the regression equations
    in the form of a dictionary which can be converted into a pandas dataframe.

    Returns: mean and diffrences between mean of control and treatment
    for all the columns whch are given as imputs (i.e characteristics)
    '''
    # create function to calculate sample standard deviation 
    sample_std_func = lambda x: x.std()/ np.sqrt(x.count())
    

    for char in dataframe.columns:
        
        # calculate mean and sample standard devaition of mean for each group
        means = dataframe.groupby(treatment)[char].mean()
        sample_std_of_mean = dataframe.groupby(treatment)[char].apply(sample_std_func)

        # run regressions with dummy variablesto calculate the difference 
        # between the means. Cluster standard errorsa at the cluster level
        # 2 treatment dummies (intercept: C) to get C-H, C-(H+S)
        diff_C_exog = sm.add_constant(
                    pd.get_dummies(dataframe[treatment], 
                    drop_first=True))

        diff_C_reg  = sm.OLS(
                            endog = dataframe[char],
                            exog  = diff_C_exog
            ).fit(cov_type='cluster', cov_kwds={'groups': dataframe[cluster]})
        
        # 2 treatment dummies (intercept: H) to get H-(H+S), H-C
        diff_C_exog = sm.add_constant(
                    pd.get_dummies(dataframe[treatment],
                    drop_first=False).drop(1.0, axis= 1) 
                    # drop dummies for H to make it reference
                    )
        
        diff_H_reg  = sm.OLS(
                            endog = dataframe[char],
                            exog  = diff_C_exog
            ).fit(cov_type='cluster', cov_kwds={'groups': dataframe[cluster]})

        # collect estimated parameters and standard errors

        if mean_or_sde == "mean":
            yield {
                "C":  means[0], 
                "H":  means[1], 
                "H + S": means[2],
                "C-H" : -diff_C_reg.params[1], 
                "C-(H + S)" : -diff_C_reg.params[2], 
                "H-(H + S)" : -diff_H_reg.params[2] 
            }
        elif mean_or_sde == "sde":
            yield {
                "C":  sample_std_of_mean[0], 
                "H":  sample_std_of_mean[1], 
                "H + S": sample_std_of_mean[2],
                "C-H" : diff_C_reg.bse[1], 
                "C-(H + S)" : diff_C_reg.bse[2], 
                "H-(H + S)" : diff_H_reg.bse[2] 
            }
        

In [25]:
# using the defined function to create a pandas dataframe, for mean and sde
# remove first 3 and last row and round the values
bm = balance_table(hh_char_clean, "treatment",  "villagecode_str",  "mean")
bal_mean = pd.DataFrame(bm, index = hh_char_clean.columns).iloc[3:-1].round(2)


bs = balance_table(hh_char_clean, "treatment",  "villagecode_str",  "sde")
bal_sde  =  pd.DataFrame(bs).iloc[3:-1].round(3)


In [26]:
def std_below_mean(mean_df, sde_df):
    ''''
    function combines the mean and std dataframes.
    first entry is the mean df and second is the std df 
    Make sure the names of the covariates are in mean_df.index 

    returns another dataframe with std below means 
    '''
    # create empty dataframe with same column names and twice as many rows
    new_df = pd.DataFrame({}, 
                          columns=mean_df.columns, 
                          index=[i  for i in  range(0,len(mean_df)*2)])
    # collect all covariate names in the list
    covariate_names = []
    # in the new database put the mean and standard dev values one after the other
    for index_12  in range(0, len(mean_df)):
        new_df.iloc[index_12*2] = mean_df.iloc[index_12]  
        new_df.iloc[(index_12*2)+1] = sde_df.iloc[index_12]
        covariate_names.append(mean_df.index[index_12])  
        covariate_names.append("_")  
    # add covariates names to the datatframe. and there we have it
    new_df.insert(0, "covariates", covariate_names, True)

    return new_df

In [27]:
baseline_covariate_balance = std_below_mean(bal_mean, bal_sde)
baseline_covariate_balance

,covariates,C,H,H + S,C-H,C-(H + S),H-(H + S)
0,hhnum_b,6.13,6.15,6.17,-0.02,-0.04,-0.02
1,_,0.076,0.075,0.072,0.134,0.132,0.133
2,hhgender_1_b,0.06,0.06,0.07,0.01,-0.01,-0.02
3,_,0.008,0.008,0.009,0.011,0.012,0.012
4,age_pc_b,34.16,33.89,33.57,0.27,0.59,0.32
5,_,0.377,0.373,0.369,0.721,0.677,0.64
6,edu_hhhead_b,0.42,0.43,0.37,-0.01,0.05,0.06
7,_,0.016,0.016,0.016,0.031,0.029,0.033
8,pc_edu_b,0.37,0.36,0.34,0.01,0.03,0.02
9,_,0.015,0.016,0.016,0.031,0.029,0.032


## Fuel Consumption Variable names baseline ###

1. Use firewood for cooking: fire_1_b
2. Use LPG for cooking: lpg_1_b
3. Use dungcakes for cooking: dung_1_b
4. Use induction stove for cooking: induction_use_b
5. Qty. of firewood purchased last month (kg): firewood_qty_b
6. Qty. of dung cakes purchased last month: dung_qty_b
7. Have LPG connection: hhscr_8_b
8. Total no. of LPG refills (annual):  totrefills_omc_b2
9. No. of LPG refills per month (winter): monthly_winb
10. No. of LPG refills per month (summer): monthly_sumb
11. No. of LPG refills per month (monsoon): monthly_wetb

In [28]:
# there are null values in the LPG data
df.groupby("treatment")["totrefills_omc_b2"].apply(lambda x: sum(x.isna()))
df.groupby("treatment")["dung_qty_b"].mean()

treatment
0.0    20.565000
1.0    37.777535
2.0    32.467970
Name: dung_qty_b, dtype: float64

In [29]:
hh_fuelcons = df[["villagecode_str","treatment","tehsil_e", "fire_1_b", "lpg_1_b", 
                "dung_1_b", "induction_use_b", "firewood_qty_b", "dung_qty_b", "hhscr_8_b",
                "totrefills_omc_b2", "monthly_winb", "monthly_sumb", "monthly_wetb",  "att_missing"]]

# remove observations with attribution missing
hh_fuelcons_c = hh_fuelcons.loc[hh_fuelcons.att_missing == 0] 
#  drop na values
hh_fuelcons_1 = hh_fuelcons_c.dropna(inplace = False)


In [30]:
fuel_bm = balance_table(hh_fuelcons_1, "treatment", "villagecode_str",  "mean")
fuel_bal_mean = pd.DataFrame(fuel_bm, index = hh_fuelcons_1.columns).iloc[3:-1].round(2)

fuel_bs = balance_table(hh_fuelcons_1, "treatment", "villagecode_str",  "sde")
fuel_bal_sde = pd.DataFrame(fuel_bs, index = hh_fuelcons_1.columns).iloc[3:-1].round(3)

In [31]:
baseline_fuelcons_balance = std_below_mean(fuel_bal_mean, fuel_bal_sde)
baseline_fuelcons_balance

,covariates,C,H,H + S,C-H,C-(H + S),H-(H + S)
0,fire_1_b,0.75,0.74,0.76,0.02,-0.01,-0.03
1,_,0.014,0.015,0.014,0.031,0.03,0.031
2,lpg_1_b,0.71,0.76,0.73,-0.05,-0.02,0.03
3,_,0.015,0.014,0.015,0.033,0.031,0.029
4,dung_1_b,0.87,0.88,0.87,-0.01,-0.0,0.01
5,_,0.011,0.011,0.011,0.019,0.021,0.021
6,induction_use_b,0.06,0.07,0.05,-0.02,0.0,0.02
7,_,0.007,0.009,0.008,0.014,0.012,0.014
8,firewood_qty_b,9.47,15.88,12.59,-6.41,-3.13,3.28
9,_,1.742,4.122,2.422,4.72,3.3,4.922
